In [1]:
import torch
import numpy as np
import cv2
from time import time, sleep

# YoLo
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors
from ultralytics.solutions import object_counter

from typing import Dict, Iterable, Optional, Tuple

## supervision
from supervision.detection.core import Detections
from supervision.draw.color import Color
from supervision.draw.utils import draw_text
from supervision.geometry.core import Point, Position, Vector


In [2]:
class ObjectDetection:
    def __init__(self, capture_index, frame_width=640, frame_height=480):
        # default parameters
        self.capture_index = capture_index
        
        # 프레임 범위 지정
        self.frame_width = frame_width
        self.frame_height = frame_height


        # model information
        self.model = YOLO(model = "yolov8m-pose.pt")

        # visual information
        self.annotator = None
        self.start_time = 0
        self.end_time = 0

        # device information
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    # 모델 예측
    def predict(self, im0, conf= 0.5, iou=0.3, save=True, verbose=True, show=False ):
        return self.model(source = im0, 
                             conf=conf,
                             iou=iou,
                             save=save,
                             verbose=verbose,
                             show=show
                             )

    def display_fps(self, im0):
        self.end_time = time()
        fps = 1 / np.round(self.end_time - self.start_time, 2)
        text = f'FPS: {int(fps)}'
        text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1.0, 2)[0]
        gap = 10
        cv2.rectangle(im0, (20 - gap, 70 - text_size[1] - gap), (20 + text_size[0] + gap, 70 + gap), (255, 255, 255), -1)
        cv2.putText(im0, text, (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 2)

    def plot_bboxes(self, results, im0):
        class_ids = []
        box_mids = {}
        self.annotator = Annotator(im0, 3, results[0].names)
        boxes = results[0].boxes.xyxy.cpu()
        box_xywhs = results[0].boxes.xywh.numpy()
        ids = results[0].boxes.id.tolist()
        # box_xywhs = results[0].boxes
        clss = results[0].boxes.cls.cpu().tolist()
        names = results[0].names
        for box, box_xywh, id, cls in zip(boxes, box_xywhs, ids, clss):
            class_ids.append(cls)
            box_mids[id] = [box_xywh[0],box_xywh[1]]
            self.annotator.box_label(box, label=names[int(cls)], color=colors(int(cls), True))
        return im0, class_ids, box_mids

    def object_counter(self, region_points):
        counter = object_counter.ObjectCounter()
        counter.set_args(view_img=False,
                 reg_pts=region_points,
                 classes_names=self.model.names,
                 draw_tracks=True)
        return counter




    def main(self):
        
        cap = cv2.VideoCapture(self.capture_index)
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.frame_width)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.frame_height)
        # 포인트 구역은 왼쪽 위가(0,0)부터 순서로 배열됨(시계방향, 왼쪽 위부터 시작)
        # (좌상), (우상), (우하),(좌하),,  #라인으로 구현은 느립니다. ...
        inside_region_point = [(self.frame_width/3,0),(self.frame_width*2/3, 0), 
                               (self.frame_width*2/3,self.frame_height),(self.frame_width/3, self.frame_height)]
        """(0,0),
                               (self.frame_width ,0),
                               (self.frame_width ,self.frame_height/2),
                               (0,self.frame_height/2)"""
     
        inside_counter = self.object_counter(inside_region_point)
        outside_region_point = []


        
        
        human_count = 0
        frame_count = 0
        
        while True:
            in_counts, out_counts, im0 = self.model_count(cap, inside_region_point,inside_counter)
            print(in_counts, out_counts)
            cv2.imshow('YOLOv8 Detection', im0)

            
            frame_count += 1
            if cv2.waitKey(5) & 0xFF == 27:
                break
            
        
        cap.release()
        cv2.destroyAllWindows()
        

    def model_count(self,cap,inside_region_point,inside_counter ):
        self.start_time = time()
        ret, im0 = cap.read()
        
        # 기본
        # results = self.predict(im0)
        # if (results is not None) and (im0 is not None) :
        
            # im0, class_ids, cls_box = self.plot_bboxes(results, im0)   

        # 추적 카운트
        results = self.model.track(im0, persist=True, verbose=False, conf=0.5, iou=0.2)
        im0_counted = inside_counter.start_counting(im0, results)
        
        if (results is not None) and (im0_counted is not None) :
            im0, class_ids, box_mids = self.plot_bboxes(results, im0_counted)
            return inside_counter.in_counts, inside_counter.out_counts, im0
            
            # fps표시 추가하는 함수 # 느림
            # self.display_fps(im0_counted)
            # 화면에 프레임 표시
        else: 
            self.display_fps(im0)
            return inside_counter.in_counts, inside_counter.out_counts, im0

        
            

        

In [ ]:
if __name__ == "__main__":
    detector = ObjectDetection(capture_index=0,frame_width = 1920, frame_height=1080)
    detector.main()

Region Counter Initiated.
0 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
